In [27]:
import pandas as pd
import numpy as np
#from tqdm import tqdm
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel("facebook_vaccination.xlsx")

In [3]:
df = df.drop('Unnamed: 0',axis=1)

In [4]:
df.head()

,Дата,Час,Збережено,Заголовок,Дайджест тексту,Тип посту,Тональність,Автор,Підписники,Демографія,Місце публікації,Підписники місця публікації,Країна,Регіон,Місто,Сума всіх реакцій,Лайки,Коментарі,Репости
0,29.05.2021,23:53,30.05.2021 01:26,"АстраЗенека работает, 3 недели после вакцинаци...","На следующей неделе тоже пойду делать тест, де...",Коментар,Негативна,Anastasia Karbe,216,Жінка,Kyrylo Melnychuk,3583,Україна,Чернівецька область,Чернівці,0,0,0,0
1,29.05.2021,23:51,30.05.2021 09:20,0,Кейт Міддлтон услід за чоловіком вакцинувалася...,Пост,Нейтральна,Igor Stasyk,419,Чоловік,Трускавецька Міська Лікарня Friends,5721,Україна,0,0,4,4,0,1
2,29.05.2021,23:48,30.05.2021 03:31,0,"Кто ещё не укололся - ловите момент, завтра ещ...",Пост,Позитивна,Viktor Yankauskas,443,Чоловік,Viktor Yankauskas,443,Україна,Київ,Київ,1,1,0,1
3,29.05.2021,23:47,30.05.2021 00:31,0,"⚡Однією з вакцин проти COVID-19, які використо...",Репост,Позитивна,Анна Редько,125,Жінка,Анна Редько,125,Україна,Волинська область,Любомль,1,1,0,0
4,29.05.2021,23:45,30.05.2021 04:18,"У меня к вам просьба, мои расчудесные! А накид...",По поводу прививки. Мне позвонили от семейного...,Коментар,Нейтральна,Irina Levandovskaya,24,Жінка,Irina Medushevskaya,37101,Україна,Одеська область,Одеса,2,2,0,0


## Data preprocessing

In [5]:
# Filter inactive
df_active = df.groupby("Автор").filter(lambda x: len(x) > 1)

In [6]:
df_active

,Дата,Час,Збережено,Заголовок,Дайджест тексту,Тип посту,Тональність,Автор,Підписники,Демографія,Місце публікації,Підписники місця публікації,Країна,Регіон,Місто,Сума всіх реакцій,Лайки,Коментарі,Репости
1,29.05.2021,23:51,30.05.2021 09:20,0,Кейт Міддлтон услід за чоловіком вакцинувалася...,Пост,Нейтральна,Igor Stasyk,419,Чоловік,Трускавецька Міська Лікарня Friends,5721,Україна,0,0,4,4,0,1
2,29.05.2021,23:48,30.05.2021 03:31,0,"Кто ещё не укололся - ловите момент, завтра ещ...",Пост,Позитивна,Viktor Yankauskas,443,Чоловік,Viktor Yankauskas,443,Україна,Київ,Київ,1,1,0,1
3,29.05.2021,23:47,30.05.2021 00:31,0,"⚡Однією з вакцин проти COVID-19, які використо...",Репост,Позитивна,Анна Редько,125,Жінка,Анна Редько,125,Україна,Волинська область,Любомль,1,1,0,0
5,29.05.2021,23:40,30.05.2021 04:46,Вчора я отримала другу дозу вакцини від covid-...,В мене все прекрасно. Не бачу причини для вакц...,Коментар,Позитивна,Maryan Lviv,17,Чоловік,Ruslana Lyzhychko,123124,Україна,Львівська область,Львів,0,0,0,0
6,29.05.2021,23:36,30.05.2021 07:42,0,ЯСЕН ПЕНЬ:\nВАКЦИН ОТ ОРЗ НЕТ И БЫТЬ НЕ ...,Пост,Нейтральна,Irina Marchenko,0,Жінка,Irina Marchenko,0,Україна,Київ,Київ,2,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332426,15.03.2021,00:00,01.04.2021 16:39,0,"мяса.\n\nВ Бейруте свет по часам, хотя пока ч...",Репост,Нейтральна,Роман Іщенко,37,Чоловік,Роман Іщенко,37,Україна,Київ,Київ,0,0,0,0
332427,15.03.2021,00:00,01.04.2021 16:39,Минздрав Чехии сокращает поставки вакцины Astr...,Сергей ПерфильевМихаил Чаленко включите гугл.,Коментар,Нейтральна,Александр Мирошник,342,Чоловік,Будни Торецка/Украина,13654,Україна,Донецька область,Торецьк,0,0,0,0
332428,15.03.2021,00:00,01.04.2021 16:39,Ранок неділі. Отримав першу дозу вакцини: Covi...,"Роман Плавицький, Даже, не стоит напрягаться.....",Коментар,Нейтральна,Sergey Stasyuk,0,Чоловік,Mathey Khrenov,6516,Україна,Житомирська область,Житомир,0,0,0,0
332429,15.03.2021,00:00,01.04.2021 16:39,Плануєте вакцинуватись від COVID-19 ?,"Сейчас около 2000 видов гриппа,вдумайтесь от ч...",Коментар,Нейтральна,Nirvana Sergeevna,925,Жінка,Los Solomas / Солом‘янський район,38051,Україна,Київ,Київ,0,0,0,0


Features needed:
1. Activity frequency
2. Out degree
3. Targets popularity
4. Katz centrality
5. Posts similarity
6. Emotional context

In [7]:
df_dropped = df_active[["Дата","Час","Дайджест тексту","Тип посту","Тональність","Автор",
                       "Підписники","Місце публікації","Підписники місця публікації"]]

In [8]:
df_dropped

,Дата,Час,Дайджест тексту,Тип посту,Тональність,Автор,Підписники,Місце публікації,Підписники місця публікації
1,29.05.2021,23:51,Кейт Міддлтон услід за чоловіком вакцинувалася...,Пост,Нейтральна,Igor Stasyk,419,Трускавецька Міська Лікарня Friends,5721
2,29.05.2021,23:48,"Кто ещё не укололся - ловите момент, завтра ещ...",Пост,Позитивна,Viktor Yankauskas,443,Viktor Yankauskas,443
3,29.05.2021,23:47,"⚡Однією з вакцин проти COVID-19, які використо...",Репост,Позитивна,Анна Редько,125,Анна Редько,125
5,29.05.2021,23:40,В мене все прекрасно. Не бачу причини для вакц...,Коментар,Позитивна,Maryan Lviv,17,Ruslana Lyzhychko,123124
6,29.05.2021,23:36,ЯСЕН ПЕНЬ:\nВАКЦИН ОТ ОРЗ НЕТ И БЫТЬ НЕ ...,Пост,Нейтральна,Irina Marchenko,0,Irina Marchenko,0
...,...,...,...,...,...,...,...,...,...
332426,15.03.2021,00:00,"мяса.\n\nВ Бейруте свет по часам, хотя пока ч...",Репост,Нейтральна,Роман Іщенко,37,Роман Іщенко,37
332427,15.03.2021,00:00,Сергей ПерфильевМихаил Чаленко включите гугл.,Коментар,Нейтральна,Александр Мирошник,342,Будни Торецка/Украина,13654
332428,15.03.2021,00:00,"Роман Плавицький, Даже, не стоит напрягаться.....",Коментар,Нейтральна,Sergey Stasyuk,0,Mathey Khrenov,6516
332429,15.03.2021,00:00,"Сейчас около 2000 видов гриппа,вдумайтесь от ч...",Коментар,Нейтральна,Nirvana Sergeevna,925,Los Solomas / Солом‘янський район,38051


In [9]:
df_renamed = df_dropped.rename(columns={"Дата":"Date","Час":"Time","Дайджест тексту":"Text","Тип посту":"Type",
                               "Тональність":"Emotional context","Автор":"Author","Підписники":"Subscribers",
                               "Місце публікації":"Target","Підписники місця публікації":"Target Subscribers"})

In [10]:
df_renamed.loc[df_renamed["Type"]=="Пост","Type"] = "Post"
df_renamed.loc[df_renamed["Type"]=="Репост","Type"] = "Repost"
df_renamed.loc[df_renamed["Type"]=="Коментар","Type"] = "Comment"

df_renamed.loc[df_renamed["Emotional context"]=="Позитивна","Emotional context"] = 1
df_renamed.loc[df_renamed["Emotional context"]=="Нейтральна","Emotional context"] = 0
df_renamed.loc[df_renamed["Emotional context"]=="Негативна","Emotional context"] = -1

In [11]:
df_processed = df_renamed
df_processed

,Date,Time,Text,Type,Emotional context,Author,Subscribers,Target,Target Subscribers
1,29.05.2021,23:51,Кейт Міддлтон услід за чоловіком вакцинувалася...,Post,0,Igor Stasyk,419,Трускавецька Міська Лікарня Friends,5721
2,29.05.2021,23:48,"Кто ещё не укололся - ловите момент, завтра ещ...",Post,1,Viktor Yankauskas,443,Viktor Yankauskas,443
3,29.05.2021,23:47,"⚡Однією з вакцин проти COVID-19, які використо...",Repost,1,Анна Редько,125,Анна Редько,125
5,29.05.2021,23:40,В мене все прекрасно. Не бачу причини для вакц...,Comment,1,Maryan Lviv,17,Ruslana Lyzhychko,123124
6,29.05.2021,23:36,ЯСЕН ПЕНЬ:\nВАКЦИН ОТ ОРЗ НЕТ И БЫТЬ НЕ ...,Post,0,Irina Marchenko,0,Irina Marchenko,0
...,...,...,...,...,...,...,...,...,...
332426,15.03.2021,00:00,"мяса.\n\nВ Бейруте свет по часам, хотя пока ч...",Repost,0,Роман Іщенко,37,Роман Іщенко,37
332427,15.03.2021,00:00,Сергей ПерфильевМихаил Чаленко включите гугл.,Comment,0,Александр Мирошник,342,Будни Торецка/Украина,13654
332428,15.03.2021,00:00,"Роман Плавицький, Даже, не стоит напрягаться.....",Comment,0,Sergey Stasyuk,0,Mathey Khrenov,6516
332429,15.03.2021,00:00,"Сейчас около 2000 видов гриппа,вдумайтесь от ч...",Comment,0,Nirvana Sergeevna,925,Los Solomas / Солом‘янський район,38051


### Creating weighted directed graph



In [14]:
df_graph = df_processed[["Author","Target"]].groupby(["Author","Target"]).size().reset_index(name="weight")

In [18]:
df_graph.head()

,Author,Target,weight
0,,,2
1,Світлана Вовк,Світлана Вовк,2
2,"""Європейська Україна""","""Європейська Україна""",7
3,"""Європейська Україна""",Чернігів UA | Перлина Полісся,1
4,"""Срібна Земля""","""Срібна Земля""",3


In [19]:
graph = nx.from_pandas_edgelist(df_graph,source="Author",target="Target",edge_attr=True,create_using=nx.DiGraph())

In [26]:
out_degrees = graph.out_degree()

In [34]:
katz_centr = nx.katz_centrality(graph.reverse(),alpha=0.1)

In [35]:
katz_centr

{' ': 0.0037729844476559467,
 ' Світлана Вовк': 0.0037729844476559467,
 '"Європейська Україна"': 0.0041502828920819725,
 'Чернігів UA | Перлина Полісся': 0.0033956860032299204,
 '"Срібна Земля"': 0.0037729844476559467,
 '032.ua - сайт мiста Львова': 0.0037729844476559467,
 '0362.ua - сайт міста Рівне': 0.0037729844476559467,
 '0412.ua - Сайт міста Житомира': 0.0037729844476559467,
 '04597.com.ua - Сайт міста Ірпеня': 0.0037729844476559467,
 '048.ua - Новости города Одессы': 0.0037729844476559467,
 '0512.com.ua - сайт города Николаева': 0.0037729844476559467,
 '0522.ua - Сайт міста Кропивницького': 0.0037729844476559467,
 '0532.ua - сайт міста Полтави - новини, афіша, довідник підприємств': 0.0037729844476559467,
 '05366.com.ua - новости, афиша, справочник предприятий': 0.0037729844476559467,
 '0564.ua - Сайт города Кривого Рога': 0.0041502828920819725,
 'Главные новости Кривого Рога': 0.0033956860032299204,
 '057.ua - Сайт города Харькова': 0.0037729844476559467,
 '061.ua - Сайт города